In [263]:
# Dependencies
import warnings
import os
from dotenv import load_dotenv
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from pydantic import BaseModel
import openai
import yfinance as yf


# Load environment variables from .env file
load_dotenv()

True

In [264]:
# Fetch API keys
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME", "text-davinci-003")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")


# Set API keys
openai.api_key = OPENAI_API_KEY

# Instantiate the tools
news_tool = SerperDevTool(api_key=SERPER_API_KEY)
scrape_tool = ScrapeWebsiteTool()

In [265]:
# Chatbot Agent
class ChatbotAgent(Agent):
    def __init__(self):
        super().__init__(
            role="Chatbot Agent",
            goal="Collect the company ticker symbol from the user",
            backstory="You are a friendly chatbot that helps users find information about companies using their ticker symbols.",
            verbose=True,
            allow_delegation=False,
        )

chatbot_agent = ChatbotAgent()


In [266]:
# News Retrieval Agent
news_retrieval_agent = Agent(
    role="News Retrieval Agent",
    goal="Fetch the latest news about the company",
    backstory="You collect the most recent news articles related to a company for sentiment analysis.",
    verbose=True,
    tools=[news_tool, scrape_tool],
)


In [267]:
# Sentiment Analysis Agent
class SentimentAnalysisAgent(Agent):
    def __init__(self):
        super().__init__(
            role="Sentiment Analysis Agent",
            goal="Analyze the sentiment of recent news articles for a given company",
            backstory="You are responsible for determining if the news sentiment about a company is positive, neutral, or negative.",
            verbose=True,
            allow_delegation=False,
        )

    def analyze_sentiment(self, articles):
        try:
            analysis_results = []
            total_score = 0
            for article in articles:
                response = openai.Completion.create(
                    engine=OPENAI_MODEL_NAME,
                    prompt=f"Score the sentiment of the following text between -1 (very negative) and 1 (very positive): \n{article}",
                    max_tokens=5,
                )
                sentiment_score = float(response.choices[0].text.strip())
                analysis_results.append(sentiment_score)
                total_score += sentiment_score

            aggregate_score = total_score / len(articles) if articles else 0
            return {"analysis_results": analysis_results, "aggregate_score": aggregate_score}
        except Exception as e:
            print(f"Error occurred during sentiment analysis: {e}")
            return {"analysis_results": [], "aggregate_score": 0}

sentiment_analysis_agent = SentimentAnalysisAgent()


In [268]:
# Define Recommendation Agent
class RecommendationAgent(Agent):
    def __init__(self):
        super().__init__(
            role="Recommendation Agent",
            goal="Provide an investment recommendation based on the sentiment analysis results",
            backstory="You provide a recommendation to the user based on the sentiment analysis of the company's recent news articles.",
            verbose=True,
            allow_delegation=False,
        )

    def generate_recommendation(self, aggregate_score):
        try:
            if aggregate_score > 0.3:
                return f"Aggregate sentiment score: {aggregate_score:.2f} - Positive news sentiment—consider investment."
            elif aggregate_score < -0.3:
                return f"Aggregate sentiment score: {aggregate_score:.2f} - Negative news sentiment—exercise caution."
            else:
                return f"Aggregate sentiment score: {aggregate_score:.2f} - Mixed sentiment—recommend further investigation."
        except Exception as e:
            print(f"Error occurred during recommendation generation: {e}")
            return "Unable to generate recommendation."

recommendation_agent = RecommendationAgent()


In [269]:
# Tasks
class NewsArticles(BaseModel):
    articles: list[str]

retrieve_news_task = Task(
    description="Fetch recent financial news articles about {company_name}",
    expected_output="A list of recent news articles.",
    output_json=NewsArticles,
    agent=news_retrieval_agent,
    tools=[news_tool, scrape_tool],
)

sentiment_analysis_task = Task(
    description="Analyze the sentiment of the news articles for {company_name}",
    expected_output="A list of sentiment analysis results and an aggregate sentiment score.",
    agent=sentiment_analysis_agent,
)

recommendation_task = Task(
    description="Provide an investment recommendation based on the sentiment analysis results", 
    expected_output="An investment recommendation for {company_name}",
    agent=recommendation_agent,
)


In [270]:
# Define the Crew and Assign Tasks
financial_analysis_crew = Crew(
    agents=[chatbot_agent, news_retrieval_agent, sentiment_analysis_agent, recommendation_agent],
    tasks=[retrieve_news_task, sentiment_analysis_task, recommendation_task],
    verbose=True,
)


2024-10-24 12:54:21,031 - 140704492328576 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [271]:
# Initiate the Process
stock_ticker = input("Please enter the company ticker symbol: ").upper()
if stock_ticker:
    print(f"Stock ticker provided: {stock_ticker}")
    result = financial_analysis_crew.kickoff(inputs={"company_name": stock_ticker})
    if result:
        sentiment_analysis_result = result.get("sentiment_analysis_task")
        recommendation_result = result.get("recommendation_task")

        if sentiment_analysis_result:
            aggregate_score = sentiment_analysis_result.get('aggregate_score', 0)
            print(f"Final Aggregate Sentiment Score: {aggregate_score:.2f}")
        if recommendation_result:
            print(recommendation_result)
else:
    print("Invalid ticker symbol. Please try again.")

Stock ticker provided: TSLA
# Agent: News Retrieval Agent
## Task: Fetch recent financial news articles about TSLA


# Agent: News Retrieval Agent
## Thought: I need to search for recent financial news articles specifically related to Tesla (TSLA).
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"recent financial news articles about TSLA\"}"
## Tool Output: 

Search results: Title: Tesla, Inc. (TSLA) Latest Stock News & Headlines - Yahoo Finance
Link: https://finance.yahoo.com/quote/TSLA/news/
Snippet: Electric vehicle pioneer Tesla (NASDAQ:TSLA) fell short of the market's revenue expectations in Q3 CY2024, but sales rose 7.8% year on year to $25.18 billion.
---
Title: Tesla shares surge as analysts react to Q3 earnings, Musk predictions
Link: https://www.cnbc.com/2024/10/24/tesla-shares-surge-as-analysts-react-to-q3-earnings-musk-predictions.html
Snippet: Tesla shares soared roughly 19% Thursday morning, putting the stock on pace for its best day in more than t

AttributeError: 'CrewOutput' object has no attribute 'task_output'